In [1]:
#Import

#pandas
import pandas as pd
from pandas import Series,DataFrame

#numpy
import numpy as np

In [2]:
#导入数据

dorm_feature=pd.read_csv('dorm_input/dorm_train.csv')
dorm_feature.head()

,id,dorm_date,out_or_in,date,year,month,hour,weekday,dorm_hour00,dorm_hour01,...,dorm_hour23,dorm_hour_01_05,dorm_hour_morning,dorm_hour_afternoon,dorm_hour_evening,dorm_hour_midnight,dorm_resting_day,dorm_working_day,dorm_on_school,dorm_vacation
0,13126,2014-01-21 03:31:11,1,01-21-2014,2014,v,a,wd,0,0,...,0,1,0,0,0,0,0,1,0,1
1,13126,2014-01-21 04:53:55,0,01-21-2014,2014,v,a,wd,0,0,...,0,1,0,0,0,0,0,1,0,1
2,18484,2014-01-21 05:16:18,1,01-21-2014,2014,v,a,wd,0,0,...,0,1,0,0,0,0,0,1,0,1
3,296,2014-01-21 05:55:05,1,01-21-2014,2014,v,a,wd,0,0,...,0,1,0,0,0,0,0,1,0,1
4,9760,2014-01-21 08:44:53,1,01-21-2014,2014,v,b,wd,0,0,...,0,0,1,0,0,0,0,1,0,1


In [3]:
#计算进出寝室的总次数

dorm_feature_f=DataFrame(dorm_feature.groupby('id')['id'].count())
dorm_feature_f.columns=['dorm_count_sum']

In [4]:
#计算活跃的天数

dorm_day=DataFrame(dorm_feature.groupby(['id','date'])['date'].count())
dorm_day.to_csv('dorm_input/dorm_day.csv',index=True)
dorm_day_count=pd.read_csv('dorm_input/dorm_day.csv')
dorm_day_count=DataFrame(dorm_day_count.groupby('id')['date.1'].count())
dorm_day_count.columns=['dorm_daycount']

In [5]:
#连接表

dorm_feature_f=dorm_feature_f.join(dorm_day_count)

In [6]:
#根据id聚合求解各个特征的产生的总数

count_columns=list(dorm_feature.columns)[8:]

#count_columns
count_pivot=dorm_feature.pivot_table(index='id',values=count_columns,aggfunc=np.count_nonzero)
dorm_feature_f=dorm_feature_f.join(count_pivot)

In [7]:
#根据id聚合求解各个特征占比(各特征进出总次数占比)

#a=['id','date','year']
mean_columns=[x for x in list(dorm_feature.columns)[8:]]
mean_pivot=dorm_feature.pivot_table(index='id',values=mean_columns,aggfunc=np.mean)

#对列名重命名
mean_pivot.columns=[x+'_rate' for x in list(mean_pivot.columns)]
dorm_feature_f=dorm_feature_f.join(mean_pivot)

In [8]:
#定义id关于每一列的平均进出次数(每天)

def get_colunms_mean(df_sum,df_count):
    a=list(df_count.columns)
    columns_mean=DataFrame(df_count[a[0]])#初始化表格大小
    columns_mean.columns=['id_count']
    for x in range(len(a)):
        columns_count_x=df_count[a[x]].values
        columns_sum=df_sum['dorm_daycount'].values
        columns_mean[a[x]+'_day_mean']=np.divide.reduce([columns_count_x,columns_sum],axis=0)
        x+=1
    columns_mean.drop('id_count',axis=1,inplace=True)
    return columns_mean

#运用定义的函数
dorm_columns_mean=get_colunms_mean(dorm_feature_f,count_pivot)

#连接总表
dorm_feature_f=dorm_feature_f.join(dorm_columns_mean)

In [9]:
#平均每天进出的次数

dorm_feature_f['dorm_day_mean']=dorm_feature_f['dorm_count_sum']/dorm_feature_f['dorm_daycount']

In [10]:
#导出数据

dorm_feature_f.to_csv('dorm_input/dorm_train_feature_f.csv',index='id')

In [11]:
count_pivot.head()

,dorm_hour00,dorm_hour01,dorm_hour02,dorm_hour03,dorm_hour04,dorm_hour05,dorm_hour06,dorm_hour07,dorm_hour08,dorm_hour09,...,dorm_hour23,dorm_hour_01_05,dorm_hour_afternoon,dorm_hour_evening,dorm_hour_midnight,dorm_hour_morning,dorm_on_school,dorm_resting_day,dorm_vacation,dorm_working_day
id,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,0,0,0,0,3,2,1,...,0,0,95,68,10,13,171,48,15,138
1,0,0,0,0,0,0,2,0,5,2,...,0,0,162,82,15,53,308,72,4,240
10,0,0,0,0,0,0,0,0,11,3,...,0,0,44,39,11,41,131,23,4,112
19,0,0,0,0,0,0,0,1,11,6,...,1,0,70,95,71,30,255,66,11,200
20,1,0,0,0,0,0,0,0,0,0,...,1,0,7,0,5,1,12,4,1,9


In [12]:
len(count_pivot)

7834

In [13]:
mean_pivot.head()

,dorm_hour00_rate,dorm_hour01_rate,dorm_hour02_rate,dorm_hour03_rate,dorm_hour04_rate,dorm_hour05_rate,dorm_hour06_rate,dorm_hour07_rate,dorm_hour08_rate,dorm_hour09_rate,...,dorm_hour23_rate,dorm_hour_01_05_rate,dorm_hour_afternoon_rate,dorm_hour_evening_rate,dorm_hour_midnight_rate,dorm_hour_morning_rate,dorm_on_school_rate,dorm_resting_day_rate,dorm_vacation_rate,dorm_working_day_rate
id,,,,,,,,,,,,,,,,,,,,,
0,0.005376,0.0,0.0,0.0,0.0,0.0,0.00000,0.016129,0.010753,0.005376,...,0.000000,0.0,0.510753,0.365591,0.053763,0.069892,0.919355,0.258065,0.080645,0.741935
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.00641,0.000000,0.016026,0.006410,...,0.000000,0.0,0.519231,0.262821,0.048077,0.169872,0.987179,0.230769,0.012821,0.769231
10,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.081481,0.022222,...,0.000000,0.0,0.325926,0.288889,0.081481,0.303704,0.970370,0.170370,0.029630,0.829630
19,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.003759,0.041353,0.022556,...,0.003759,0.0,0.263158,0.357143,0.266917,0.112782,0.958647,0.248120,0.041353,0.751880
20,0.076923,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.000000,...,0.076923,0.0,0.538462,0.000000,0.384615,0.076923,0.923077,0.307692,0.076923,0.692308


In [14]:
dorm_feature_f.head()

,dorm_count_sum,dorm_daycount,dorm_hour00,dorm_hour01,dorm_hour02,dorm_hour03,dorm_hour04,dorm_hour05,dorm_hour06,dorm_hour07,...,dorm_hour_01_05_day_mean,dorm_hour_afternoon_day_mean,dorm_hour_evening_day_mean,dorm_hour_midnight_day_mean,dorm_hour_morning_day_mean,dorm_on_school_day_mean,dorm_resting_day_day_mean,dorm_vacation_day_mean,dorm_working_day_day_mean,dorm_day_mean
id,,,,,,,,,,,,,,,,,,,,,
0,186,58,1,0,0,0,0,0,0,3,...,0,1,1,0,0,2,0,0,2,3.206897
1,312,97,0,0,0,0,0,0,2,0,...,0,1,0,0,0,3,0,0,2,3.216495
10,135,56,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,2,2.410714
19,266,99,0,0,0,0,0,0,0,1,...,0,0,0,0,0,2,0,0,2,2.686869
20,13,9,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,1.444444
